# Week 9: 2016/03/21-25

## Monday reading

Skim Section 4.1 but take a good look at Figure 4.10.

Read the beginning of Section 4.2, up to but not including "The Diagonalization Method."

## Tuesday class

Section 4.1 is long. It shows:

- Regular, context-free, decidable, and Turing-recognizable languages form a hierarchy (Figure 4.10).
- Some results about regular and context-free languages that are interesting in their own right, like the fact that you can test whether two DFAs are equivalent. (We used this fact to partially automate the grading of one exam question.)
- A more subtle point is that _a priori_ we might think that some context-free languages are not decidable. For a PDA can have a computation branch that goes on forever. But in fact, it's possible to simulate the PDA so that infinite computation branches can be detected and cut short.

### The universal Turing machine

Our focus for today is the universal TM $U$ defined at the beginning of the proof of Theorem 4.11. It is a TM that can read in the description of another TM and simulate it. For some reason, the book deemphasizes this idea. It's significant not only because of its relevance to Theorem 4.11, but also because it gave rise to (or at least anticipated) the idea of a stored program computer.

First, we have to show how to encode a TM as a string. This is easy to do and the details are not important, but it's worth at least mentioning one way to do it for concreteness. Turing's original encoding used an alphabet $\{\mathtt{A}, \mathtt{C}, \mathtt{D}, \mathtt{L}, \mathtt{R}, \mathtt{N}\}$. If the states in $Q$ are numbered $q_1, q_2, \ldots$, then state $q_i$ is encoded as $\mathtt{DA}^i$. If the symbols in $\Gamma$ are numbered $a_0, a_1, \ldots$, where $a_0$ is the blank symbol, then symbol $a_i$ is encoded as $\mathtt{DC}^i$. Then, the transition $\delta(q_i, a_j) = (q_k, a_\ell, \textrm{L})$ is encoded as $\mathtt{DA}^i \mathtt{DC}^j \mathtt{DC}^\ell \mathtt{L} \mathtt{DA}^k$, and similarly if the move is R or N. We also need some convention for indicating the start, accept, and reject states: perhaps $q_1$ is always the start state, $q_2$ the accept state, and $q_3$ the reject state.

Second, we have to show the universal TM itself. It is often constructed as a TM with three tapes:

1. An encoding of $M$, the machine being simulated.
2. The tape of $M$.
3. The state of $M$.

An implementation-level description would be: On input $\langle M, w\rangle$, where $M$ is a TM and $w$ is a string:

1. Copy $M$ onto tape 1 and $w$ onto tape 2.
2. Initialize tape 3 to the start state of $M$.
3. Repeat:
   1. If the state (tape 3) is the accept state, _accept_; if it is the reject state, _reject_.
   2. Search on tape 1 for an instruction that matches the current state (encoded on tape 3) and current input symbol (encoded on tape 2).
   3. Write the new state to tape 3 and the new symbol to tape 2.
   4. Move the head on tape 2 to the left or right as indicated by the instruction.

### Miscellany

There's a cottage industry of seeing who can make the smallest universal TM. The [current record holder](http://computerage-onthesamepage.berkeley.edu/tiny-turing-machine), due to Rogozhin, has only a couple dozen instructions.

## Wednesday reading

Read Section 4.2 and, for fun, the poem "[Scooping the Loop Snooper](http://www.lel.ed.ac.uk/~gpullum/loopsnoop.html): A proof that the Halting Problem is undecidable."

## Thursday class

### Countable and uncountable

The book has a long digression about countable and uncountable infinities. If you are not familiar with these concepts and Cantor's diagonalization argument, pay careful attention to this part of the section, even though it may not seem relevant at first. The point of this digression is that the vast majority of languages are _not_ Turing-recognizable (or decidable).

### An undecidable language

The next subsection gives an actual example of an undecidable language. In the 2nd edition of the book, this subsection (beginning on page 179) is misleadingly called "The Halting Problem." But the language $A_{\mathsf{TM}}$ is _not_ the halting problem, and this subsection was retitled "An Undecidable Language" in the 3rd edition.

What would it mean for $A_{\mathsf{TM}}$ to be decidable? If $A_{\mathsf{TM}}$ were decidable, then that would mean we could simulate any non-decider TM in such a way that looping could be avoided (as is the case for PDAs).

### A simpler proof

The proof that $A_{\mathsf{TM}}$ is undecidable is a mind-bender. Perhaps this version of the proof is conceptually easier (similar to the proof in Hopcroft and Ullman).

Suppose that there exists a decider $H$ that decides $A_{\mathsf{TM}}$. Then we can imagine a big table of all possible outcomes of $H$ (assuming an alphabet $\{\mathtt{0}, \mathtt{1}\}$ for illustration's sake):

| | $\varepsilon$ | $\mathtt{0}$ |$\mathtt{1}$ | $\mathtt{00}$ | $\cdots$ |
|-|-------------|------------|-------------|-------------|--------|
|$M_1$| _reject_ | _accept_ | _reject_ | _accept_    |        |
|$M_2$| _reject_ | _accept_ | _accept_ | _reject_    |        |
|$M_3$| _accept_ | _reject_ | _accept_ | _reject_ |  |
|$M_4$| _reject_ | _reject_ | _reject_ | _accept_ |                |
|$\vdots$| | | | | | |

We should be able to compute the value of any cell of this table since $H$ is supposedly a decider. Therefore, we should be able to compute a function $D$ that is the opposite of every diagonal entry of this table. But this is a contradiction, because $D$ must itself have a row in the table, yet it differs in at least one column from every row in the table.

More precisely, let $M_i$ be the $i$th machine in some ordering of Turing machines, and let $w_i$ be the $i$th string in some ordering of strings in $\Sigma^\ast$ (page 14). Then $D$ is defined as follows. On input $w$, 

1. Find the least $i$ such that $w = w_i$. 
2. Run $H$ on input $\langle M_i, w_i \rangle$.
3. If $H$ accepts, _reject_, and if $H$ rejects, _accept_.

To modify this version of the proof into the version in the book, observe that we can choose any ordering of the $w_i$ we want. So choose an ordering such that the $i$th string happens to be the encoding of the $i$th machine, that is, $w_i = \langle M_i \rangle$. (The fact that this ordering misses some strings doesn't matter -- the proof still goes through.)

### A Python demo

We can also demonstrate the argument in any programming language, here Python. Assume that we have a function called `accepts` that takes two arguments, `function` and `input`, and supposedly can decide whether `function` accepts `input` (even if `function` does not halt on `input`).

In [1]:
def accepts(function, input):
    """Returns True if function(input) is True.
       Returns False if function(input) is False or does not halt."""
    # Note: this code is nonsense. You can replace it with whatever you want.
    return (hash(function.__code__.co_code) + hash(input)) % 2 == 0

Then we can design a function and an input that "breaks" `accepts`.

In [2]:
def evil_function(input):
    if callable(input):
        return not accepts(input, input)
    else:
        return False # arbitrary return value for non-callables
evil_input = evil_function

Let's test it out! When we use `accepts` to test whether `evil_function` accepts `evil_input`, we get:

In [3]:
accepts(evil_function, evil_input)

True

But the correct answer is:

In [4]:
evil_function(evil_input)

False

### Turing-recognizable and co-Turing-recognizable

The last subsection exhibits a language that is not even Turing-recognizable. It is, frankly, not that interesting. The more important thing we learn here is Theorem 4.22,

\begin{align*}
\text{decidable} &= \text{Turing-recognizable} \cap \text{co-Turing-recognizable}.
\end{align*}

### Postscript

Have a blessed Easter Triduum!